# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import tiktoken
import base64
import io
import json
from pdf2image import convert_from_path
import PyPDF2
import os
import base64
import requests

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.all import *

In [ ]:
#| export
def encode_image(image_path):
  """Function to encode the image"""
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Test
image_path = '../example_data/1.jpg'
image_base64 = encode_image(image_path)
test_eq_type(type(image_base64), str)

In [ ]:
#| export
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

In [ ]:
# test
image_path = '../example_data/1.jpg'
base64_image = encode_image(image_path)
messages = [{"role": "user", "content": "Hello, how are you?", "image": base64_image}]
num_tokens = num_tokens_from_messages(messages)
num_tokens


351620

In [ ]:
#| export
DEFAULT_SYSTEM_PROMPT = """You are a helpful assistant designed to see an exam and output JSON \ 
with the extracted information. You will be given an image of the exam."""

DEFAULT_USER_PROMPT ="""Extract the last name (Apellidos in Spanish) and the first name (Nombre in Spanish) \
 from the top of the image. You will find them handwritten after the labels `Apellidos` \
 and `Nombre` respectively. The fields of your JSON output will have those exact same label names"""

In [ ]:
#| export
def extract_student_info(path_pdf: str, crop=None, api_key = None, 
                 model="gpt-4o", prompt=DEFAULT_USER_PROMPT, 
                 system_prompt=DEFAULT_SYSTEM_PROMPT, verbose=False):
    """
    Extracts student information from a PDF file using OpenAI's chatGPT.

    Args:
        path_pdf (str): The path to the PDF file.
        crop (tuple, optional): Tuple with the coordinates of the crop (left, top, right, bottom).
        api_key (str, optional): The API key for accessing the OpenAI API. If 
            not provided, the function will try to use the OPENAI_API_KEY environment variable.
        model (str, optional): The model to use for generating responses.
        prompt (str, optional): The user prompt to start the conversation.
        system_prompt (str, optional): The system prompt to provide context to the model.
        verbose (bool, optional): Whether to print information about the process.

    Returns:
        str: The extracted information from the PDF.

    Raises:
        KeyError: If the OPENAI_API_KEY environment variable is not set.
        IndexError: If the response from the OpenAI API does not contain any choices.

    """
    with open(path_pdf, 'rb') as file:
        pdf = PyPDF2.PdfFileReader(file)

        # Check if the PDF has more than one page
        if pdf.getNumPages() > 1 and verbose:
            print("The PDF has more than one page. Only the first page will be \
                  converted to an image.")
        
        # Convert the PDF to images and get the first one
        image = convert_from_path(path_pdf, last_page=1, first_page=0, fmt='jpeg')[0]

        # Crop the image
        if crop:
            image = image.crop(crop)
        
        # Convert to base64
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ifnone(api_key, os.environ.get('OPENAI_API_KEY'))}"
        }
        payload = {
            "model": f'{model}',
            "response_format": { "type": "json_object" },
            "messages": [
                {
                    "role": "system", 
                    "content": f"{system_prompt}",   
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": f"{prompt}"
                    },
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                    }
                ]
                }
            ],
            "max_tokens": 300
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", 
                                 headers=headers, json=payload)
        
        return json.loads(response.json()['choices'][0]['message']['content'])

In [ ]:
# test
path_pdf = '../example_data/1.pdf'
info = extract_student_info(path_pdf)
info

The PDF has more than one page. Only the first page will be                   converted to an image.


'{\n  "Apellidos": "RODRIGUEZ FERNANDEZ",\n  "Nombre": "VICTOR"\n}'

In [ ]:
rename_exam_file?

Signature:
rename_exam_file(
    pdf_path: str,
    output_path,
    *,
    crop=None,
    api_key=None,
    model='gpt-4o',
    prompt='Extract the last name (Apellidos in Spanish) and the first name (Nombre in Spanish)\n from the top of the image. You will find them handwritten after the labels `Apellidos` \n and `Nombre` respectively. The fields of your JSON output will have those exact same label names',
    system_prompt='You are a helpful assistant designed to see an exam and output JSON     with the extracted information. You will be given an image of the exam.',
    verbose=False,
)
Docstring:
Extracts student information from an exam PDF file using OpenAI's chatGPT and renames that
file with the extracted information as the name.

Parameters:
- pdf_path (str): The path to the input PDF file.
- output_path (str): The path to the directory where the renamed file will be saved.
- **kwargs: Additional keyword arguments to be passed to the `extract_student_info` function.

Returns:

In [ ]:
#| export
@delegates(extract_student_info)
def rename_exam_file(pdf_path: str, output_path, **kwargs):
    """
    Extracts student information from an exam PDF file using OpenAI's chatGPT and renames that
    file with the extracted information as the name.

    Parameters:
    - pdf_path (str): The path to the input PDF file.
    - output_path (str): The path to the directory where the renamed file will be saved.
    - **kwargs: Additional keyword arguments to be passed to the `extract_student_info` function.

    Returns:
    - new_name (str): The new name of the renamed file.

    Example:
    >>> rename_exam_file('/path/to/input.pdf', '/path/to/output', option1='value1', option2='value2')
    'Doe_John.pdf'
    """
    info = extract_student_info(pdf_path, **kwargs)
    new_name = f"{info['Apellidos']}_{info['Nombre']}.pdf"
    os.rename(pdf_path, os.path.join(output_path, new_name))
    return new_name

In [ ]:
# Test
path_pdf = '../example_data/1.pdf'
output_path = '../example_data/'
new_name = rename_exam_file

In [ ]:
#| export
def rename_file(folder_path: str, output_path='./exam_sort_ocr', **kwargs):
    """
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for file in os.listdir(folder_path):
        if file.endswith('.pdf'):
            path_pdf = os.path.join(folder_path, file)
            info = extract_info(path_pdf, **kwargs)
            info['path'] = path_pdf
            with open(os.path.join(output_path, file.replace('.pdf', '.json')), 'w') as f:
                json.dump(info, f)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()